In [1]:
# import the libraries
import cv2
from openvino.inference_engine import IECore
import numpy as np

# Load the model
ie = IECore()

net = ie.read_network(
    model = "face-detection-adas-0001.xml",
    weights = "face-detection-adas-0001.bin"
)

model = ie.load_network(net,'CPU')

# initiate camera object
cam = cv2.VideoCapture(0)

# do the loop
while True:
    # read the frame
    ret,frame = cam.read()

    # resize and reshape the image
    resized = cv2.resize(frame,(672,384))
    input_data = np.expand_dims(np.transpose(resized,(2,0,1)),0).astype(np.float32)

    # do the inference and draw the bounding box
    result = model.infer({'data':input_data})
    for i in result['detection_out'][0][0]:
        if i[2]>0.7:
            h,w,c = frame.shape
            cv2.rectangle(
                frame,
                (int(i[3]*w),int(i[4]*h)),
                (int(i[5]*w),int(i[6]*h)),
                (0,255,0),4
            )
    # show the image in a windows
    cv2.imshow('Face Detection (Press q to exit)',frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# destroy the camera object and close all windows
cam.release()
cv2.destroyAllWindows()